# 🏀 DoubleDunk - DDQN Optimizado para MAIA

**Reto de Aprendizaje por Refuerzo Profundo**  
**Algoritmo:** Double Deep Q-Network (DDQN) con Epsilon Scheduler Cíclico  
**Entorno:** ALE/DoubleDunk-v5 (Atari Basketball)  
**Plataforma:** Google Colab con GPU optimizado para entrenamiento largo  



## 📋 Información del Proyecto

- **Curso:** Aprendizaje por Refuerzo Profundo - MAIA
- **Problema:** Optimización de agente para juego DoubleDunk
- **Método:** DDQN con mejoras específicas vs REINFORCE baseline
- **Tiempo estimado:** 6-12 horas en GPU Colab (con interrupciones)


⚠️ **IMPORTANTE:** Este notebook está diseñado para entrenamientos largos en GPU. Utiliza checkpoints automáticos para manejar desconexiones de Colab.


## 📦 Instalación de Dependencias (Google Colab GPU)

**Optimizado para sesiones GPU largas con gestión de memoria**


In [ ]:
# ========================================
# INSTALACIÓN OPTIMIZADA PARA GPU COLAB
# ========================================

import os
import sys
import subprocess

print("🚀 Configurando entorno para DDQN DoubleDunk en GPU...")

# Verificar que estamos en Colab
try:
    import google.colab
    IN_COLAB = True
    print("✅ Google Colab detectado")
except ImportError:
    IN_COLAB = False
    print("⚠️  Ejecutándose fuera de Colab")

# Instalaciones principales con verificación de errores
packages = [
    'stable-baselines3[extra]',
    'ale-py',
    'gymnasium[atari,accept-rom-license]',
    'autorom',
    'tensorboard',
    'opencv-python',
    'imageio[ffmpeg]',
    'pandas',
    'matplotlib',
    'seaborn',
    'tqdm'
]

for package in packages:
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', package])
        print(f"✅ {package}")
    except subprocess.CalledProcessError as e:
        print(f"❌ Error instalando {package}: {e}")

# Configurar ROMs de Atari
try:
    subprocess.run(['AutoROM', '--accept-license'], check=True, 
                  capture_output=True, text=True)
    print("✅ ROMs de Atari configuradas")
except:
    print("⚠️  ROMs ya configuradas o error menor")

# Configurar directorio de trabajo
if IN_COLAB:
    os.makedirs('/content/ddqn_doubledunk', exist_ok=True)
    os.chdir('/content/ddqn_doubledunk')
    print("📁 Directorio de trabajo: /content/ddqn_doubledunk")

print("✅ Configuración completa - Listo para GPU")
print("🎯 Iniciando importaciones...")


In [ ]:
# ========================================
# IMPORTACIONES Y CONFIGURACIÓN GPU
# ========================================

# Librerías RL y utilidades
import stable_baselines3
from stable_baselines3 import DQN
from stable_baselines3.common.logger import configure
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import BaseCallback, CallbackList
from stable_baselines3.common.vec_env import VecMonitor, VecFrameStack, VecVideoRecorder
from stable_baselines3.common.env_util import make_atari_env

import gymnasium
import ale_py
from collections import deque
import cv2

# Registrar entornos ALE
gymnasium.register_envs(ale_py)

# Librerías básicas
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import random
import math
import time
import json
import pickle
import glob
import base64
from datetime import datetime, timedelta
from tqdm import tqdm
from IPython.display import HTML, display, clear_output

# PyTorch para GPU
import torch
from torch.nn import functional as F

print("📚 Importaciones completadas")
print(f"🔢 Stable Baselines3: {stable_baselines3.__version__}")
print(f"🔥 PyTorch: {torch.__version__}")


## 🖥️ Detección y Configuración de GPU


In [ ]:
# ========================================
# DETECCIÓN AUTOMÁTICA DE GPU COLAB
# ========================================

import platform
import psutil

def detect_colab_hardware():
    """
    Detecta y configura el hardware disponible en Google Colab
    Optimizado para GPU T4, P100, V100, A100
    """
    print("=" * 60)
    print("🖥️  DETECCIÓN DE HARDWARE GOOGLE COLAB")
    print("=" * 60)
    
    # Información del sistema
    print(f"Sistema: {platform.system()} {platform.release()}")
    print(f"CPU: {platform.processor()}")
    print(f"RAM: {psutil.virtual_memory().total / 1024**3:.1f} GB")
    print(f"PyTorch: {torch.__version__}")
    
    device_info = {}
    
    # Verificar CUDA (GPU)
    if torch.cuda.is_available():
        device = torch.device('cuda')
        gpu_name = torch.cuda.get_device_name(0)
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
        gpu_compute = torch.cuda.get_device_properties(0).major
        
        device_info = {
            'device': device,
            'name': gpu_name,
            'memory_gb': gpu_memory,
            'compute_capability': gpu_compute,
            'type': 'GPU_CUDA',
            'recommended': True
        }
        
        print(f"✅ GPU DETECTADA: {gpu_name}")
        print(f"   💾 Memoria GPU: {gpu_memory:.1f} GB")
        print(f"   🔧 Compute Capability: {gpu_compute}.x")
        
        # Configuraciones específicas por GPU
        if 'T4' in gpu_name:
            print(f"   🎯 Tesla T4 detectada - Configuración optimizada")
            batch_size_factor = 1.0
        elif 'P100' in gpu_name:
            print(f"   🚀 Tesla P100 detectada - Configuración de alto rendimiento")
            batch_size_factor = 1.2
        elif 'V100' in gpu_name:
            print(f"   💎 Tesla V100 detectada - Configuración premium")
            batch_size_factor = 1.5
        elif 'A100' in gpu_name:
            print(f"   🌟 Tesla A100 detectada - Configuración máxima")
            batch_size_factor = 2.0
        else:
            print(f"   🔧 GPU genérica detectada - Configuración estándar")
            batch_size_factor = 1.0
            
        device_info['batch_size_factor'] = batch_size_factor
        
    else:
        # Fallback a CPU
        device = torch.device('cpu')
        device_info = {
            'device': device,
            'name': 'CPU',
            'type': 'CPU',
            'recommended': False,
            'batch_size_factor': 0.5
        }
        print(f"⚠️  SOLO CPU DISPONIBLE")
        print(f"   ❌ No se detectó GPU - El entrenamiento será MUY lento")
        print(f"   💡 Asegúrate de activar GPU en Colab: Runtime > Change runtime type > GPU")
    
    print("-" * 60)
    print(f"DISPOSITIVO FINAL: {device_info['name']} ({device_info['device']})")
    
    if device_info['recommended']:
        print(f"✅ Configuración óptima para entrenamiento largo")
    else:
        print(f"⚠️  ADVERTENCIA: Sin GPU el entrenamiento puede tomar días")
    
    print("=" * 60)
    
    return device_info['device'], device_info

# Detectar hardware
DEVICE, DEVICE_INFO = detect_colab_hardware()

# Configuraciones de PyTorch para GPU
if DEVICE.type == 'cuda':
    print("🚀 Aplicando optimizaciones CUDA...")
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False
    # Limpiar caché de GPU
    torch.cuda.empty_cache()
    print(f"   📊 Memoria GPU inicial: {torch.cuda.memory_allocated()/1024**3:.2f} GB")
else:
    print("💻 Configurando para CPU...")
    torch.set_num_threads(4)  # Limitar threads en Colab

print(f"\n🎯 Sistema configurado para: {DEVICE_INFO['name']}")


## 💾 Sistema de Checkpoints Inteligente

**Gestión automática de checkpoints para entrenamientos largos con interrupciones**


In [ ]:
# ========================================
# SISTEMA DE CHECKPOINTS INTELIGENTE
# ========================================

class CheckpointManager:
    """
    Gestor de checkpoints optimizado para entrenamientos largos en Colab
    Maneja desconexiones automáticamente y preserva todo el estado
    """
    
    def __init__(self, checkpoint_dir="./checkpoints", backup_every=50000):
        self.checkpoint_dir = checkpoint_dir
        self.backup_every = backup_every
        self.session_start = datetime.now()
        
        # Crear directorios
        os.makedirs(checkpoint_dir, exist_ok=True)
        os.makedirs(f"{checkpoint_dir}/models", exist_ok=True)
        os.makedirs(f"{checkpoint_dir}/training_state", exist_ok=True)
        
        print(f"💾 Checkpoint Manager inicializado")
        print(f"   📁 Directorio: {checkpoint_dir}")
        print(f"   ⏰ Backup cada: {backup_every:,} timesteps")
    
    def _convert_to_json_serializable(self, obj):
        """
        Convierte objetos a tipos serializables por JSON
        """
        if isinstance(obj, dict):
            return {key: self._convert_to_json_serializable(value) for key, value in obj.items()}
        elif isinstance(obj, list):
            return [self._convert_to_json_serializable(item) for item in obj]
        elif hasattr(obj, 'item'):  # numpy scalars
            return obj.item()
        elif hasattr(obj, 'tolist'):  # numpy arrays
            return obj.tolist()
        elif isinstance(obj, (np.int32, np.int64, np.float32, np.float64)):
            return float(obj) if 'float' in str(type(obj)) else int(obj)
        else:
            return obj
    
    def save_checkpoint(self, model, timestep, episode_rewards, metadata=None):
        """
        Guarda checkpoint completo del estado del entrenamiento
        """
        try:
            # Convertir episode_rewards a float estándar para JSON serialization
            safe_episode_rewards = [float(reward) for reward in episode_rewards]
            
            # Convertir metadata recursivamente
            safe_metadata = self._convert_to_json_serializable(metadata or {})
            
            checkpoint_data = {
                'timestep': int(timestep),
                'timestamp': datetime.now().isoformat(),
                'session_duration': str(datetime.now() - self.session_start),
                'episode_rewards': safe_episode_rewards,
                'device': str(DEVICE),
                'metadata': safe_metadata
            }
            
            # Guardar modelo
            model_path = f"{self.checkpoint_dir}/models/ddqn_checkpoint_{timestep}.zip"
            model.save(model_path)
            
            # Guardar estado de entrenamiento
            state_path = f"{self.checkpoint_dir}/training_state/state_{timestep}.json"
            with open(state_path, 'w') as f:
                json.dump(checkpoint_data, f, indent=2)
            
            print(f"💾 Checkpoint guardado: timestep {timestep:,}")
            return True
            
        except Exception as e:
            print(f"❌ Error guardando checkpoint: {e}")
            return False
    
    def list_checkpoints(self):
        """Lista todos los checkpoints disponibles"""
        checkpoints = []
        model_files = glob.glob(f"{self.checkpoint_dir}/models/ddqn_checkpoint_*.zip")
        
        for model_file in model_files:
            try:
                timestep = int(model_file.split('_')[-1].split('.')[0])
                state_file = f"{self.checkpoint_dir}/training_state/state_{timestep}.json"
                
                if os.path.exists(state_file):
                    with open(state_file, 'r') as f:
                        state_data = json.load(f)
                    
                    checkpoints.append({
                        'timestep': timestep,
                        'model_path': model_file,
                        'state_path': state_file,
                        'timestamp': state_data.get('timestamp', 'Unknown'),
                        'episodes': len(state_data.get('episode_rewards', [])),
                        'last_reward': state_data.get('episode_rewards', [0])[-1] if state_data.get('episode_rewards') else 0
                    })
            except:
                continue
        
        return sorted(checkpoints, key=lambda x: x['timestep'])
    
    def get_latest_checkpoint(self):
        """Obtiene el checkpoint más reciente"""
        checkpoints = self.list_checkpoints()
        return checkpoints[-1] if checkpoints else None
    
    def load_checkpoint(self, timestep=None):
        """Carga un checkpoint específico o el más reciente"""
        if timestep is None:
            checkpoint = self.get_latest_checkpoint()
        else:
            checkpoints = self.list_checkpoints()
            checkpoint = next((c for c in checkpoints if c['timestep'] == timestep), None)
        
        if checkpoint is None:
            return None, None
        
        try:
            # Cargar estado
            with open(checkpoint['state_path'], 'r') as f:
                state_data = json.load(f)
            
            print(f"📂 Cargando checkpoint: timestep {checkpoint['timestep']:,}")
            print(f"   📅 Fecha: {checkpoint['timestamp']}")
            print(f"   📊 Episodios: {checkpoint['episodes']}")
            print(f"   🎯 Última recompensa: {checkpoint['last_reward']:.2f}")
            
            return checkpoint['model_path'], state_data
            
        except Exception as e:
            print(f"❌ Error cargando checkpoint: {e}")
            return None, None

# Inicializar gestor de checkpoints
checkpoint_manager = CheckpointManager(
    checkpoint_dir="./checkpoints_doubledunk",
    backup_every=50000
)

# Verificar checkpoints existentes
existing_checkpoints = checkpoint_manager.list_checkpoints()
if existing_checkpoints:
    print(f"\n📋 Checkpoints existentes encontrados: {len(existing_checkpoints)}")
    for cp in existing_checkpoints[-3:]:  # Mostrar los 3 más recientes
        print(f"   ⏰ {cp['timestep']:,} steps - {cp['timestamp'][:19]} - Reward: {cp['last_reward']:.2f}")
else:
    print(f"\n📋 No se encontraron checkpoints - Entrenamiento desde cero")

print(f"\n✅ Sistema de checkpoints configurado")


## 🧠 Algoritmo DDQN Optimizado + Callbacks + Trainer

**Implementación completa con sistema de checkpoints integrado**


In [ ]:
# ========================================
# IMPLEMENTACIÓN COMPLETA DDQN + SISTEMA CHECKPOINT
# ========================================

# Callbacks optimizados
class RewardLoggerCallback(BaseCallback):
    def __init__(self, path_backup: str = None, checkpoint_manager=None, usar_media: bool = True, ventana_para_media: int = 30):
        super().__init__(verbose=True)
        self.episode_rewards = []
        self.best_score = -np.inf
        self.path_backup = path_backup
        self.checkpoint_manager = checkpoint_manager
        self.usar_media = usar_media
        self.ventana_para_media = ventana_para_media
        self.last_checkpoint_step = 0

    def _on_step(self) -> bool:
        infos = self.locals.get("infos", [])
        if not infos:
            return True

        for info in infos:
            ep = info.get("episode")
            if ep is None:
                continue
            r = ep.get("r")
            self.episode_rewards.append(r)

            # Checkpoint automático cada 50k steps
            if (self.checkpoint_manager and 
                self.num_timesteps - self.last_checkpoint_step >= self.checkpoint_manager.backup_every):
                self.checkpoint_manager.save_checkpoint(
                    self.model, self.num_timesteps, self.episode_rewards,
                    {'best_score': self.best_score, 'total_episodes': len(self.episode_rewards)}
                )
                self.last_checkpoint_step = self.num_timesteps

            # Evaluar mejor modelo
            if self.usar_media:
                if len(self.episode_rewards) >= self.ventana_para_media:
                    score = float(np.mean(self.episode_rewards[-self.ventana_para_media:]))
                else:
                    score = float(np.mean(self.episode_rewards))
            else:
                score = float(r)

            if score > self.best_score:
                self.best_score = score
                if self.path_backup:
                    dirname = os.path.dirname(self.path_backup)
                    if dirname and not os.path.exists(dirname):
                        os.makedirs(dirname, exist_ok=True)
                    try:
                        self.model.save(self.path_backup)
                        print(f"🏆 Nuevo mejor score {score:.2f} -> Guardado en: {self.path_backup}.zip")
                    except Exception as e:
                        print(f"❌ Error guardando mejor modelo: {e}")
        return True

class EpsilonSchedulerCallback(BaseCallback):
    def __init__(self, schedule_fn, total_timesteps: int, verbose: int = 1):
        super().__init__(verbose)
        self.schedule_fn = schedule_fn
        self.total_timesteps = int(total_timesteps)

    def _on_step(self) -> bool:
        t = min(self.num_timesteps, self.total_timesteps)
        progress = t / float(self.total_timesteps)
        new_eps = float(self.schedule_fn(progress))
        self.model.exploration_rate = new_eps

        if ((self.num_timesteps - 1) % 10_000 == 0):
            try:
                self.logger.record("train/epsilon", new_eps)
                lr = float(self.model.policy.optimizer.param_groups[0]["lr"])
                self.logger.record("train/learning_rate", lr)
                if self.verbose:
                    print(f"📊 Step {self.num_timesteps:,} | ε={new_eps:.4f} | LR={lr:.6f}")
            except Exception:
                pass
        return True

# Epsilon scheduler
def crear_eps_scheduler(val_inicial: float, val_min: float, n_ciclos: int, degree: int):
    def scheduler(progress: float) -> float:
        envelope = (1.0 - progress**degree)
        cos_term = 0.5 * (1.0 + np.cos(2 * np.pi * n_ciclos * progress))
        val = val_inicial * envelope * cos_term
        return float(max(val, val_min))
    return scheduler

# DDQN implementación
class OptimizedDoubleDQN(DQN):
    def train(self, gradient_steps: int, batch_size: int = 32) -> None:
        self.policy.set_training_mode(True)
        self._update_learning_rate(self.policy.optimizer)
        
        losses = []
        for _ in range(gradient_steps):
            replay_data = self.replay_buffer.sample(batch_size, env=self._vec_normalize_env)
            obs = replay_data.observations
            next_obs = replay_data.next_observations
            actions = replay_data.actions
            rewards = replay_data.rewards
            dones = replay_data.dones

            if rewards.dim() == 1:
                rewards = rewards.unsqueeze(1)
            if dones.dim() == 1:
                dones = dones.unsqueeze(1)
            if actions.dim() == 1:
                actions = actions.unsqueeze(1)

            device = self.device
            obs = obs.to(device)
            next_obs = next_obs.to(device)
            actions = actions.to(device)
            rewards = rewards.to(device).float()
            dones = dones.to(device).float()

            with torch.no_grad():
                q_next_online = self.q_net(next_obs)
                next_actions_online = q_next_online.argmax(dim=1, keepdim=True)
                q_next_target = self.q_net_target(next_obs)
                q_next_target_selected = torch.gather(q_next_target, dim=1, index=next_actions_online)
                target_q_values = rewards + (1.0 - dones) * (self.gamma * q_next_target_selected)

            q_values_all = self.q_net(obs)
            current_q_values = torch.gather(q_values_all, dim=1, index=actions.long())
            
            loss = F.huber_loss(current_q_values, target_q_values, delta=1.0)
            losses.append(loss.item())
            
            self.policy.optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.policy.parameters(), self.max_grad_norm * 0.5)
            self.policy.optimizer.step()

        self._n_updates += gradient_steps
        self.logger.record("train/n_updates", self._n_updates, exclude="tensorboard")
        self.logger.record("train/loss", np.mean(losses))

# Configuración optimizada para GPU
DOUBLEDUNK_GPU_CONFIG = {
    'total_timesteps': 5_000_000,
    'n_ciclos_eps': 12,
    'eps_inicial': 0.95,
    'eps_min': 0.01,
    'scheduler_degree': 1.5,
    'ventana_media': 30,
    'learning_rate': 2.5e-4 if DEVICE_INFO['recommended'] else 1e-4,
    'buffer_size': 200_000,
    'batch_size': int(32 * DEVICE_INFO.get('batch_size_factor', 1.0)),
    'target_update': 8_000,
    'train_freq': 4,
    'learning_starts': 20_000,
    'gamma': 0.995,
}

print("⚙️ Configuración GPU optimizada:")
for key, value in DOUBLEDUNK_GPU_CONFIG.items():
    print(f"   {key}: {value}")

def create_optimized_gpu_model(env):
    return OptimizedDoubleDQN(
        "CnnPolicy",
        env,
        learning_rate=DOUBLEDUNK_GPU_CONFIG['learning_rate'],
        buffer_size=DOUBLEDUNK_GPU_CONFIG['buffer_size'],
        learning_starts=DOUBLEDUNK_GPU_CONFIG['learning_starts'],
        batch_size=DOUBLEDUNK_GPU_CONFIG['batch_size'],
        gradient_steps=1,
        gamma=DOUBLEDUNK_GPU_CONFIG['gamma'],
        train_freq=DOUBLEDUNK_GPU_CONFIG['train_freq'],
        target_update_interval=DOUBLEDUNK_GPU_CONFIG['target_update'],
        policy_kwargs=dict(
            net_arch=[512, 256] if DEVICE_INFO['recommended'] else [256, 128],
            activation_fn=torch.nn.ReLU,
            normalize_images=True
        ),
        tensorboard_log="./logs_doubledunk",
        verbose=1,
        device=DEVICE,
    )

print("✅ Algoritmo DDQN y callbacks configurados")


## 🚀 Entrenamiento con Checkpoints Automáticos

**Entrenamiento largo optimizado para GPU con sistema de recuperación**


In [ ]:
# ========================================
# ENTRENAMIENTO CON SISTEMA DE CHECKPOINTS
# ========================================

def setup_training_with_checkpoints():
    """Configura entrenamiento con capacidad de reanudación"""
    
    # Verificar si hay checkpoint existente
    model_path, state_data = checkpoint_manager.load_checkpoint()
    
    if model_path and state_data:
        # Reanudar desde checkpoint
        print("🔄 REANUDANDO ENTRENAMIENTO DESDE CHECKPOINT")
        print(f"   📅 Última sesión: {state_data['timestamp']}")
        print(f"   📊 Timesteps completados: {state_data['timestep']:,}")
        print(f"   🎯 Episodios: {len(state_data['episode_rewards'])}")
        
        # Crear entorno
        env = make_atari_env("ALE/DoubleDunk-v5", n_envs=1, seed=0)
        env = VecFrameStack(env, n_stack=4)
        env = VecMonitor(env, filename="./logs_doubledunk/monitor.csv")
        
        # Cargar modelo existente
        model = OptimizedDoubleDQN.load(model_path, env=env, device=DEVICE)
        
        # Configurar callbacks con estado existente
        scheduler = crear_eps_scheduler(
            DOUBLEDUNK_GPU_CONFIG['eps_inicial'], 
            DOUBLEDUNK_GPU_CONFIG['eps_min'], 
            DOUBLEDUNK_GPU_CONFIG['n_ciclos_eps'], 
            DOUBLEDUNK_GPU_CONFIG['scheduler_degree']
        )
        
        eps_cb = EpsilonSchedulerCallback(
            lambda p: scheduler(p), 
            total_timesteps=DOUBLEDUNK_GPU_CONFIG['total_timesteps']
        )
        
        reward_cb = RewardLoggerCallback(
            path_backup="./checkpoints_doubledunk/best_model",
            checkpoint_manager=checkpoint_manager,
            ventana_para_media=DOUBLEDUNK_GPU_CONFIG['ventana_media']
        )
        
        # Restaurar estado de rewards
        reward_cb.episode_rewards = state_data['episode_rewards']
        reward_cb.best_score = state_data['metadata'].get('best_score', -np.inf)
        
        # Calcular timesteps restantes
        remaining_timesteps = DOUBLEDUNK_GPU_CONFIG['total_timesteps'] - state_data['timestep']
        
        return model, env, [reward_cb, eps_cb], remaining_timesteps
        
    else:
        # Entrenamiento desde cero
        print("🆕 INICIANDO ENTRENAMIENTO DESDE CERO")
        
        # Crear entorno
        env = make_atari_env("ALE/DoubleDunk-v5", n_envs=1, seed=0)
        env = VecFrameStack(env, n_stack=4)
        env = VecMonitor(env, filename="./logs_doubledunk/monitor.csv")
        
        # Crear modelo nuevo
        model = create_optimized_gpu_model(env)
        
        # Configurar logger
        new_logger = configure("./logs_doubledunk", ["csv", "tensorboard"])
        model.set_logger(new_logger)
        
        # Configurar callbacks
        scheduler = crear_eps_scheduler(
            DOUBLEDUNK_GPU_CONFIG['eps_inicial'], 
            DOUBLEDUNK_GPU_CONFIG['eps_min'], 
            DOUBLEDUNK_GPU_CONFIG['n_ciclos_eps'], 
            DOUBLEDUNK_GPU_CONFIG['scheduler_degree']
        )
        
        eps_cb = EpsilonSchedulerCallback(
            lambda p: scheduler(p), 
            total_timesteps=DOUBLEDUNK_GPU_CONFIG['total_timesteps']
        )
        
        reward_cb = RewardLoggerCallback(
            path_backup="./checkpoints_doubledunk/best_model",
            checkpoint_manager=checkpoint_manager,
            ventana_para_media=DOUBLEDUNK_GPU_CONFIG['ventana_media']
        )
        
        return model, env, [reward_cb, eps_cb], DOUBLEDUNK_GPU_CONFIG['total_timesteps']

# Configurar entrenamiento
model, env, callbacks, timesteps_to_train = setup_training_with_checkpoints()
callback_list = CallbackList(callbacks)

print(f"🎯 Configuración de entrenamiento completada")
print(f"   🖥️  Dispositivo: {DEVICE_INFO['name']}")
print(f"   ⏱️  Timesteps a entrenar: {timesteps_to_train:,}")
print(f"   🎮 Batch size: {DOUBLEDUNK_GPU_CONFIG['batch_size']}")

# EJECUTAR ENTRENAMIENTO
print(f"\n{'='*60}")
print(f"🚀 INICIANDO ENTRENAMIENTO DDQN EN {DEVICE_INFO['name'].upper()}")
print(f"{'='*60}")

training_start = time.time()

try:
    model.learn(
        total_timesteps=timesteps_to_train,
        log_interval=5,
        callback=callback_list,
        progress_bar=True
    )
    
    training_end = time.time()
    training_duration = training_end - training_start
    
    print(f"\n✅ ENTRENAMIENTO COMPLETADO")
    print(f"   ⏱️  Duración: {training_duration/3600:.2f} horas")
    print(f"   🎯 Total episodes: {len(callbacks[0].episode_rewards)}")
    
    # Guardar modelo final
    final_model_path = "./DDQN_DoubleDunk_GPU_Final"
    model.save(final_model_path)
    print(f"   💾 Modelo final guardado: {final_model_path}")
    
    # Checkpoint final
    checkpoint_manager.save_checkpoint(
        model, 
        DOUBLEDUNK_GPU_CONFIG['total_timesteps'], 
        callbacks[0].episode_rewards,
        {
            'training_completed': True,
            'total_duration_hours': training_duration/3600,
            'final_best_score': callbacks[0].best_score
        }
    )
    
except KeyboardInterrupt:
    print(f"\n⚠️  Entrenamiento interrumpido por usuario")
    print(f"   💾 El progreso se ha guardado automáticamente en checkpoints")
except Exception as e:
    print(f"\n❌ Error durante entrenamiento: {e}")
    print(f"   💾 Revisando último checkpoint disponible...")

print(f"\n🎯 Progresando a evaluación...")


## 📊 Evaluación Completa y Generación de Resultados

**Evaluación académica con todas las evidencias requeridas**


In [ ]:
# ========================================
# EVALUACIÓN COMPLETA PARA REPORTE ACADÉMICO
# ========================================

def comprehensive_evaluation():
    """Evaluación completa del modelo entrenado"""
    
    print("📊 INICIANDO EVALUACIÓN COMPLETA")
    eval_start = time.time()
    eval_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Crear entorno de evaluación
    eval_env = make_atari_env("ALE/DoubleDunk-v5", n_envs=1, seed=42)
    eval_env = VecFrameStack(eval_env, n_stack=4)
    
    # Cargar mejor modelo
    try:
        best_model = OptimizedDoubleDQN.load("./checkpoints_doubledunk/best_model", device=DEVICE)
        print("✅ Mejor modelo cargado exitosamente")
    except:
        try:
            best_model = model  # Usar modelo en memoria si falla la carga
            print("⚠️  Usando modelo en memoria")
        except:
            print("❌ No se pudo cargar modelo para evaluación")
            return None
    
    # 1. EVALUACIÓN REQUERIDA (10 episodios)
    print("🏆 Evaluación oficial (10 episodios)...")
    mean_10, std_10 = evaluate_policy(best_model, eval_env, n_eval_episodes=10, deterministic=False, render=False)
    
    # 2. EVALUACIÓN EXTENDIDA (20 episodios)
    print("📈 Evaluación extendida (20 episodios)...")
    mean_20, std_20 = evaluate_policy(best_model, eval_env, n_eval_episodes=20, deterministic=False, render=False)
    
    eval_end = time.time()
    eval_duration = eval_end - eval_start
    
    # Obtener estadísticas de entrenamiento
    try:
        episode_rewards = callbacks[0].episode_rewards
        total_episodes = len(episode_rewards)
        best_score = callbacks[0].best_score
        
        if total_episodes > 0:
            final_100 = episode_rewards[-100:] if total_episodes >= 100 else episode_rewards
            mean_last_100 = np.mean(final_100)
            best_episode = np.max(episode_rewards)
            worst_episode = np.min(episode_rewards)
        else:
            mean_last_100 = 0
            best_episode = 0
            worst_episode = 0
    except:
        episode_rewards = []
        total_episodes = 0
        best_score = 0
        mean_last_100 = 0
        best_episode = 0
        worst_episode = 0
    
    # REPORTE OFICIAL
    print(f"{'='*70}")
    print(f"📋 REPORTE OFICIAL - DDQN DOUBLEDUNK GPU COLAB")
    
    print(f"📅 Fecha evaluación: {eval_timestamp}")
    print(f"⏱️  Tiempo evaluación: {eval_duration:.2f}s")
    print(f"🖥️  Dispositivo: {DEVICE_INFO['name']} ({DEVICE})")
    print(f"📊 Episodes entrenados: {total_episodes:,}")
    
    
    print(f"🎯 RESULTADOS PRINCIPALES:")
    
    print(f"├─ 🏆 DDQN (10 episodios):    {mean_10:.2f} ± {std_10:.2f}")
    print(f"└─ 📈 DDQN (20 episodios):    {mean_20:.2f} ± {std_20:.2f}")
    
    improvement = mean_20 - (-14.0)
    improvement_pct = (improvement / abs(-14.0)) * 100
    
    print(f"📊 ANÁLISIS DE MEJORA:")
    print(f"├─ 📶 Mejora absoluta:        {improvement:+.2f} puntos")
    print(f"├─ 📈 Mejora porcentual:      {improvement_pct:+.1f}%")
    print(f"└─ 🏅 Mejor score entrenamiento: {best_score:.2f}")
    
    
    
    # Guardar resultados estructurados
    results = {
        'experiment_info': {
            'timestamp': eval_timestamp,
            'device': str(DEVICE),
            'device_name': DEVICE_INFO['name'],
            'total_training_episodes': total_episodes
        },
        'evaluation_results': {
            'reinforce_baseline': -14.0,
            'ddqn_10_episodes': {'mean': float(mean_10), 'std': float(std_10)},
            'ddqn_20_episodes': {'mean': float(mean_20), 'std': float(std_20)},
            'improvement_absolute': float(improvement),
            'improvement_percentage': float(improvement_pct),
            'best_training_score': float(best_score)
        },
        'training_stats': {
            'mean_last_100_episodes': float(mean_last_100),
            'best_episode': float(best_episode),
            'worst_episode': float(worst_episode)
        }
    }
    
    # Exportar resultados
    with open('ddqn_doubledunk_gpu_results.json', 'w') as f:
        json.dump(results, f, indent=2)
    
    # CSV para análisis
    eval_df = pd.DataFrame({
        'Model': ['REINFORCE_baseline', 'DDQN_10eps', 'DDQN_20eps'],
        'Mean_Score': [-14.0, mean_10, mean_20],
        'Std_Score': [0.0, std_10, std_20],
        'Episodes': [10, 10, 20]
    })
    eval_df.to_csv('ddqn_doubledunk_gpu_evaluation.csv', index=False)
    
    print(f"💾 ARCHIVOS GENERADOS:")
    print(f"├─ ddqn_doubledunk_gpu_results.json")
    print(f"└─ ddqn_doubledunk_gpu_evaluation.csv")
    
    eval_env.close()
    return results

# Ejecutar evaluación
evaluation_results = comprehensive_evaluation()

# Mostrar progreso de entrenamiento si está disponible
try:
    if len(callbacks[0].episode_rewards) > 10:
        plt.figure(figsize=(15, 6))
        
        plt.subplot(1, 2, 1)
        rewards = callbacks[0].episode_rewards
        plt.plot(rewards, alpha=0.6, color='blue')
        if len(rewards) > 50:
            window = 50
            moving_avg = pd.Series(rewards).rolling(window=window).mean()
            plt.plot(moving_avg, color='red', linewidth=2, label=f'Media móvil ({window})')
        plt.axhline(y=-14.0, color='orange', linestyle='--', label='REINFORCE baseline')
        plt.xlabel('Episodios')
        plt.ylabel('Recompensa')
        plt.title('Evolución del Entrenamiento DDQN')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        plt.subplot(1, 2, 2)
        plt.hist(rewards, bins=30, alpha=0.7, color='green', edgecolor='black')
        plt.axvline(x=-14.0, color='orange', linestyle='--', label='REINFORCE baseline')
        plt.xlabel('Recompensa')
        plt.ylabel('Frecuencia')
        plt.title('Distribución de Recompensas')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        print("📈 Gráficas de entrenamiento generadas")
except:
    print("⚠️  No se pudieron generar gráficas (entrenamiento muy corto o datos no disponibles)")


In [ ]:
# ========================================
# FUNCIÓN DE GENERACIÓN DE VIDEOS
# ========================================

import cv2
import imageio
from PIL import Image
import base64
from IPython.display import HTML, display

def generate_video_best_model(
    video_filename="ddqn_doubledunk_best_model.mp4",
    num_episodes=3,
    max_steps_per_episode=2000,
    fps=30,
    seed=42
):
    """
    Genera un video del mejor modelo jugando DoubleDunk episodios completos
    
    Args:
        video_filename: Nombre del archivo de video a generar
        num_episodes: Número de episodios a grabar
        max_steps_per_episode: Máximo de pasos por episodio
        fps: Frames por segundo del video
        seed: Semilla para reproducibilidad
    """
    print(f"🎬 GENERANDO VIDEO DEL MEJOR MODELO")
    print(f"=" * 50)
    
    # Verificar si existe el mejor modelo
    best_model_path = "./checkpoints_doubledunk/best_model.zip"
    if not os.path.exists(best_model_path):
        print(f"❌ No se encontró el mejor modelo en: {best_model_path}")
        print("💡 Asegúrate de que el entrenamiento haya generado un mejor modelo")
        return None
    
    try:
        # Cargar el mejor modelo
        print(f"📂 Cargando mejor modelo desde: {best_model_path}")
        model = DQN.load(best_model_path)
        print(f"✅ Modelo cargado exitosamente")
        
        # Crear ambiente para grabación (sin VecVideoRecorder para mejor control)
        print(f"🎮 Configurando ambiente DoubleDunk...")
        env = make_atari_env("ALE/DoubleDunk-v5", n_envs=1, seed=seed)
        env = VecFrameStack(env, n_stack=4)
        
        # Configurar grabación manual con mejor control
        video_folder = "./videos/"
        os.makedirs(video_folder, exist_ok=True)
        
        print(f"🎬 Iniciando grabación:")
        print(f"   📹 Episodios: {num_episodes}")
        print(f"   ⏱️  Max pasos por episodio: {max_steps_per_episode}")
        print(f"   🎯 Semilla: {seed}")
        print(f"   📁 Carpeta: {video_folder}")
        
        # Variables para estadísticas y frames
        episode_rewards = []
        episode_lengths = []
        all_frames = []
        total_steps = 0
        
        # Ejecutar episodios y recopilar frames
        for episode in range(num_episodes):
            print(f"   🎮 Iniciando episodio {episode + 1}/{num_episodes}...")
            
            obs = env.reset()
            episode_reward = 0
            episode_length = 0
            episode_frames = []
            
            for step in range(max_steps_per_episode):
                # Obtener frame antes de la acción
                # Para vectorized env, necesitamos obtener el frame del environment interno
                try:
                    # Intentar obtener frame del environment interno
                    if hasattr(env, 'render'):
                        frame = env.render()
                    else:
                        # Para VecEnv, acceder al environment base
                        frame = env.envs[0].render()
                    
                    if frame is not None:
                        episode_frames.append(frame)
                except:
                    # Si no podemos obtener frame, continuamos sin él
                    pass
                
                # Predecir acción usando el modelo
                action, _ = model.predict(obs, deterministic=True)
                
                # Ejecutar acción
                obs, reward, done, info = env.step(action)
                
                episode_reward += reward[0]
                episode_length += 1
                total_steps += 1
                
                # Si el episodio terminó
                if done[0]:
                    # Obtener frame final
                    try:
                        if hasattr(env, 'render'):
                            final_frame = env.render()
                        else:
                            final_frame = env.envs[0].render()
                        if final_frame is not None:
                            episode_frames.append(final_frame)
                    except:
                        pass
                    break
            
            # Guardar estadísticas del episodio
            episode_rewards.append(episode_reward)
            episode_lengths.append(episode_length)
            all_frames.extend(episode_frames)
            
            print(f"   🏁 Episodio {episode + 1}: Reward={episode_reward:.2f}, Steps={episode_length}, Frames={len(episode_frames)}")
        
        # Cerrar ambiente
        env.close()
        
        # Estadísticas finales
        if episode_rewards:
            mean_reward = np.mean(episode_rewards)
            std_reward = np.std(episode_rewards)
            mean_length = np.mean(episode_lengths)
            
            print(f"\n📊 ESTADÍSTICAS DE GRABACIÓN:")
            print(f"   🏆 Recompensa promedio: {mean_reward:.2f} ± {std_reward:.2f}")
            print(f"   📏 Duración promedio: {mean_length:.1f} pasos")
            print(f"   🎯 Total de pasos: {total_steps}")
            print(f"   🎬 Total de frames: {len(all_frames)}")
            print(f"   ⏱️  Episodios completados: {num_episodes}")
        
        # Generar video si tenemos frames
        if all_frames:
            final_video_path = os.path.join(video_folder, video_filename)
            
            print(f"\n🎬 Generando video con {len(all_frames)} frames...")
            
            # Usar imageio para crear el video
            imageio.mimsave(
                final_video_path, 
                all_frames, 
                fps=fps,
                quality=8,
                macro_block_size=1  # Evitar problemas de resolución
            )
            
            print(f"\n✅ VIDEO GENERADO EXITOSAMENTE:")
            print(f"   📁 Ubicación: {final_video_path}")
            print(f"   📊 Tamaño: {os.path.getsize(final_video_path) / 1024 / 1024:.1f} MB")
            print(f"   ⏱️  Duración: ~{len(all_frames) / fps:.1f} segundos")
            
            return final_video_path
        else:
            print(f"❌ No se pudieron capturar frames para el video")
            print(f"💡 Esto puede deberse a problemas de renderizado en el ambiente")
            return None
            
    except Exception as e:
        print(f"❌ Error generando video: {e}")
        import traceback
        traceback.print_exc()
        return None

def generate_video_robust_method(
    video_filename="ddqn_doubledunk_best_model.mp4",
    num_episodes=3,
    max_steps_per_episode=10000,  # Aumentar límite para episodios completos
    fps=30,
    seed=42
):
    """
    Método robusto que garantiza episodios completos de DoubleDunk
    Usa el ambiente exacto del entrenamiento para compatibilidad total
    """
    print(f"🎬 GENERANDO VIDEO - MÉTODO ROBUSTO")
    print(f"=" * 50)
    
    # Verificar si existe el mejor modelo
    best_model_path = "./checkpoints_doubledunk/best_model.zip"
    if not os.path.exists(best_model_path):
        print(f"❌ No se encontró el mejor modelo en: {best_model_path}")
        return None
    
    try:
        # Cargar el mejor modelo
        print(f"📂 Cargando mejor modelo desde: {best_model_path}")
        model = DQN.load(best_model_path)
        print(f"✅ Modelo cargado exitosamente")
        
        # Crear ambiente EXACTAMENTE igual al entrenamiento para compatibilidad total
        print(f"🎮 Configurando ambiente EXACTO del entrenamiento...")
        
        # Usar make_atari_env como en el entrenamiento pero para un solo env
        env_single = make_atari_env("ALE/DoubleDunk-v5", n_envs=1, seed=seed)
        env_single = VecFrameStack(env_single, n_stack=4)
        
        # Para la grabación, también crear un ambiente de renderizado
        import gymnasium as gym
        render_env = gym.make("ALE/DoubleDunk-v5", render_mode="rgb_array")
        
        # Aplicar los mismos wrappers que make_atari_env
        from stable_baselines3.common.atari_wrappers import (
            NoopResetEnv, MaxAndSkipEnv, EpisodicLifeEnv, 
            FireResetEnv, WarpFrame, ClipRewardEnv
        )
        
        render_env = NoopResetEnv(render_env, noop_max=30)
        render_env = MaxAndSkipEnv(render_env, skip=4)
        render_env = EpisodicLifeEnv(render_env)
        if "FIRE" in render_env.unwrapped.get_action_meanings():
            render_env = FireResetEnv(render_env)
        render_env = WarpFrame(render_env)
        render_env = ClipRewardEnv(render_env)
        
        # Frame stacking manual para sincronizar
        from collections import deque
        frame_stack = deque(maxlen=4)
        
        video_folder = "./videos/"
        os.makedirs(video_folder, exist_ok=True)
        
        print(f"🎬 Iniciando grabación con método robusto:")
        print(f"   📹 Episodios: {num_episodes}")
        print(f"   ⏱️  Max pasos por episodio: {max_steps_per_episode}")
        print(f"   🎯 Semilla: {seed}")
        print(f"   🔄 Usando ambiente exacto del entrenamiento")
        
        # Variables para estadísticas y frames
        episode_rewards = []
        episode_lengths = []
        all_frames = []
        
        # Ejecutar episodios con doble ambiente (predicción + renderizado)
        for episode in range(num_episodes):
            print(f"   🎮 Iniciando episodio {episode + 1}/{num_episodes}...")
            
            # Reset ambos ambientes con la misma semilla
            obs_model = env_single.reset()  # Para el modelo DDQN
            obs_render, info_render = render_env.reset(seed=seed + episode)  # Para renderizado
            
            # Inicializar frame stack para el ambiente de renderizado
            frame_stack.clear()
            for _ in range(4):
                frame_stack.append(obs_render)
            
            episode_reward = 0
            episode_length = 0
            episode_frames = []
            
            # Variables para sincronización
            render_done = False
            model_done = False
            
            for step in range(max_steps_per_episode):
                # Obtener frame para video desde ambiente de renderizado
                if not render_done:
                    frame = render_env.render()
                    if frame is not None:
                        episode_frames.append(frame)
                
                # Predecir acción usando el ambiente del modelo (exacto al entrenamiento)
                if not model_done:
                    action, _ = model.predict(obs_model, deterministic=True)
                    
                    # Ejecutar en ambiente del modelo
                    obs_model, reward_model, done_model, info_model = env_single.step(action)
                    model_done = done_model[0]
                    episode_reward += reward_model[0]
                    
                    # Ejecutar la misma acción en ambiente de renderizado (sincronizado)
                    if not render_done:
                        obs_render, reward_render, terminated, truncated, info_render = render_env.step(action[0])
                        render_done = terminated or truncated
                        frame_stack.append(obs_render)
                
                episode_length += 1
                
                # Terminar cuando cualquiera de los dos termine
                if model_done or render_done:
                    print(f"     💡 Episodio terminó: Model_done={model_done}, Render_done={render_done}")
                    # Obtener frame final si es posible
                    if not render_done:
                        try:
                            final_frame = render_env.render()
                            if final_frame is not None:
                                episode_frames.append(final_frame)
                        except:
                            pass
                    break
            
            # Guardar estadísticas
            episode_rewards.append(episode_reward)
            episode_lengths.append(episode_length)
            all_frames.extend(episode_frames)
            
            print(f"   🏁 Episodio {episode + 1}: Reward={episode_reward:.2f}, Steps={episode_length}, Frames={len(episode_frames)}")
            
            # Verificar si el episodio fue muy corto (posible problema)
            if episode_length < 10:
                print(f"     ⚠️  Episodio muy corto - posible problema de sincronización")
        
        # Cerrar ambientes
        env_single.close()
        render_env.close()
        
        # Estadísticas finales
        if episode_rewards:
            mean_reward = np.mean(episode_rewards)
            std_reward = np.std(episode_rewards)
            mean_length = np.mean(episode_lengths)
            
            print(f"\n📊 ESTADÍSTICAS DE GRABACIÓN:")
            print(f"   🏆 Recompensa promedio: {mean_reward:.2f} ± {std_reward:.2f}")
            print(f"   📏 Duración promedio: {mean_length:.1f} pasos")
            print(f"   🎬 Total de frames: {len(all_frames)}")
            print(f"   ⏱️  Episodios completados: {num_episodes}")
        
        # Generar video
        if all_frames:
            final_video_path = os.path.join(video_folder, video_filename)
            
            print(f"\n🎬 Generando video con {len(all_frames)} frames...")
            
            imageio.mimsave(
                final_video_path, 
                all_frames, 
                fps=fps,
                quality=8,
                macro_block_size=1
            )
            
            print(f"\n✅ VIDEO GENERADO EXITOSAMENTE:")
            print(f"   📁 Ubicación: {final_video_path}")
            print(f"   📊 Tamaño: {os.path.getsize(final_video_path) / 1024 / 1024:.1f} MB")
            print(f"   ⏱️  Duración: ~{len(all_frames) / fps:.1f} segundos")
            
            return final_video_path
        else:
            print(f"❌ No se pudieron capturar frames")
            return None
            
    except Exception as e:
        print(f"❌ Error en método alternativo: {e}")
        import traceback
        traceback.print_exc()
        return None

def diagnose_video_issue(video_path):
    """
    Diagnostica problemas comunes en la generación de videos
    """
    print(f"🔍 DIAGNÓSTICO DEL VIDEO")
    print(f"=" * 30)
    
    if not os.path.exists(video_path):
        print(f"❌ El video no existe: {video_path}")
        return
    
    # Información básica del archivo
    file_size = os.path.getsize(video_path)
    print(f"📁 Archivo: {video_path}")
    print(f"📊 Tamaño: {file_size / 1024:.1f} KB")
    
    # Intentar leer con imageio
    try:
        reader = imageio.get_reader(video_path)
        frame_count = reader.count_frames()
        meta = reader.get_meta_data()
        fps = meta.get('fps', 30)
        duration = frame_count / fps if fps > 0 else 0
        
        print(f"🎬 Frames: {frame_count}")
        print(f"⏱️  FPS: {fps}")
        print(f"⏰ Duración: {duration:.2f} segundos")
        
        # Leer algunos frames para diagnóstico
        sample_frames = min(5, frame_count)
        print(f"🖼️  Muestreando {sample_frames} frames...")
        
        for i in range(sample_frames):
            try:
                frame = reader.get_data(i)
                print(f"   Frame {i}: {frame.shape} - {frame.dtype}")
            except Exception as e:
                print(f"   Frame {i}: Error - {e}")
        
        reader.close()
        
        # Diagnóstico de problemas comunes
        if duration < 5:
            print(f"⚠️  VIDEO MUY CORTO - Posibles causas:")
            print(f"   • Episodios terminan muy rápido")
            print(f"   • Modelo no está funcionando correctamente")
            print(f"   • Problema de ambiente/wrappers")
        
        if frame_count < 100:
            print(f"⚠️  POCOS FRAMES - Posibles causas:")
            print(f"   • Captura de frames fallando")
            print(f"   • Renderizado no funciona")
            print(f"   • Sincronización de ambientes")
        
        print(f"✅ Diagnóstico completado")
        
    except Exception as e:
        print(f"❌ Error leyendo video: {e}")
        print(f"💡 El archivo puede estar corrupto o en formato incorrecto")

def display_video_in_notebook(video_path):
    """
    Muestra el video directamente en el notebook de Jupyter/Colab
    """
    if not os.path.exists(video_path):
        print(f"❌ Video no encontrado: {video_path}")
        return
    
    try:
        # Leer y encodear video en base64
        with open(video_path, "rb") as f:
            video_data = f.read()
        
        video_base64 = base64.b64encode(video_data).decode()
        
        # Crear HTML para mostrar video
        video_html = f"""
        <div style="text-align: center; margin: 20px;">
            <h3>🎬 DDQN DoubleDunk - Mejor Modelo</h3>
            <video width="640" height="480" controls style="border: 2px solid #4CAF50; border-radius: 10px;">
                <source src="data:video/mp4;base64,{video_base64}" type="video/mp4">
                Tu navegador no soporta videos HTML5.
            </video>
            <p style="margin-top: 10px; color: #666;">
                📁 Archivo: {os.path.basename(video_path)} | 
                📊 Tamaño: {os.path.getsize(video_path) / 1024 / 1024:.1f} MB
            </p>
        </div>
        """
        
        display(HTML(video_html))
        print(f"✅ Video mostrado en el notebook")
        
    except Exception as e:
        print(f"❌ Error mostrando video: {e}")
        print(f"💡 Puedes descargar el video directamente desde: {video_path}")

# Información sobre la generación de videos
print("🎬 SISTEMA DE GENERACIÓN DE VIDEOS MEJORADO")
print("=" * 50)
print("Funciones disponibles:")
print("   generate_video_robust_method()       # Método principal (ambientes sincronizados)")
print("   generate_video_best_model()         # Método de respaldo")
print("   display_video_in_notebook()         # Mostrar video en notebook")
print()
print("✨ MEJORAS IMPLEMENTADAS:")
print("   🎯 Episodios completos - No se corta en medio del juego")
print("   🔄 Ambientes sincronizados - Modelo + Renderizado en paralelo")
print("   📹 Captura frame por frame - Más confiable")
print("   🎮 Ambiente exacto - Misma configuración del entrenamiento")
print("   ⏱️  Duración precisa - Control total del video")
print("   🛡️  Detección de problemas - Diagnóstico automático")
print()
print("💡 El video se genera automáticamente usando el mejor modelo guardado")
print("   durante el entrenamiento (best_model.zip)")


In [ ]:
# ========================================
# EJECUTAR GENERACIÓN DE VIDEO
# ========================================

# Configuración de video
VIDEO_CONFIG = {
    'video_filename': 'ddqn_doubledunk_best_model.mp4',
    'num_episodes': 10,           # Número de episodios a grabar
    'max_steps_per_episode': 2000,  # Pasos máximos por episodio
    'seed': 42                   # Semilla para reproducibilidad
}

print("🎬 CONFIGURACIÓN DE VIDEO:")
print("=" * 30)
for key, value in VIDEO_CONFIG.items():
    print(f"   {key}: {value}")
print()

# Generar video del mejor modelo - Intentar método alternativo primero
print("🚀 Iniciando generación de video...")
print("💡 Probando método alternativo (mejor control de episodios completos)...")

video_path = generate_video_robust_method(**VIDEO_CONFIG)

# Si el método alternativo falla, intentar método original
if video_path is None:
    print("\n🔄 Método alternativo falló, probando método original...")
    video_path = generate_video_best_model(**VIDEO_CONFIG)

if video_path:
    print(f"\n🎉 ¡Video generado exitosamente!")
    print(f"📁 Ubicación: {video_path}")
    
    
    
    # Información adicional para el reporte académico
    print(f"\n📋 INFORMACIÓN PARA EL REPORTE:")
    print(f"✅ Video evidencia disponible en: {video_path}")
    print(f"🎯 Agente entrenado con DDQN + Epsilon Scheduler")
    print(f"🏆 Modelo usado: Mejor modelo durante entrenamiento")
    print(f"🎮 Ambiente: ALE/DoubleDunk-v5")
    print(f"🎲 Semilla: {VIDEO_CONFIG['seed']} (reproducible)")
    print(f"📊 Episodios grabados: {VIDEO_CONFIG['num_episodes']}")
    
else:
    print(f"\n❌ No se pudo generar el video")
    print(f"💡 Posibles causas:")
    print(f"   - El modelo no existe (entrenamiento no completado)")
    print(f"   - Error de ambiente o dependencias")
    print(f"   - Falta de espacio en disco")
    print(f"\n🔧 Soluciones:")
    print(f"   1. Verificar que existe: ./checkpoints_doubledunk/best_model.zip")
    print(f"   2. Ejecutar el entrenamiento antes de generar video")
    print(f"   3. Revisar los logs de error arriba")
